<a href="https://colab.research.google.com/github/Reinaldo-34/sentimentGoogle/blob/main/googleReadV2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [86]:
import pandas as pd
import matplotlib.pyplot as plt

from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials
gc = gspread.authorize(GoogleCredentials.get_application_default())

In [87]:
wb = gc.open_by_url('https://docs.google.com/spreadsheets/d/1_vROEMAQ29haCkI1yowbZh1h5qJfFWctnOqhxHneytE/edit#gid=0')

import nltk 
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [88]:
sheet = wb.worksheet('Sheet1')
data = sheet.get_all_values()
df = pd.DataFrame(data)
df.columns = df.iloc[0]
df = df.iloc[1:]

In [89]:
reviewList = list(df['Review text'])


In [107]:
print(reviewList[0])
print(range(len(reviewList)))

Emmanuel was extremely helpful and supportive throughout the entire property purchase process. As a first home buyer, there were many things I was unaware of and Emmanuel took the time to explain how it all works and how to assess properties to truly make the most of my investment. Emmanuel even went out of his way to help me investigate properties beyond the ones I originally contacted him about so I could feel confident in my decision - he was patient, supportive and empathetic from initial contact to exchange. Thank you!
range(0, 63)


In [109]:
## iterate through all items in ReviewList and tokenize
tokens = []

for i in range(len(reviewList)):
  x = nltk.word_tokenize(reviewList[i])
  tokens.append(x)



[['Emmanuel', 'was', 'extremely', 'helpful', 'and', 'supportive', 'throughout', 'the', 'entire', 'property', 'purchase', 'process', '.', 'As', 'a', 'first', 'home', 'buyer', ',', 'there', 'were', 'many', 'things', 'I', 'was', 'unaware', 'of', 'and', 'Emmanuel', 'took', 'the', 'time', 'to', 'explain', 'how', 'it', 'all', 'works', 'and', 'how', 'to', 'assess', 'properties', 'to', 'truly', 'make', 'the', 'most', 'of', 'my', 'investment', '.', 'Emmanuel', 'even', 'went', 'out', 'of', 'his', 'way', 'to', 'help', 'me', 'investigate', 'properties', 'beyond', 'the', 'ones', 'I', 'originally', 'contacted', 'him', 'about', 'so', 'I', 'could', 'feel', 'confident', 'in', 'my', 'decision', '-', 'he', 'was', 'patient', ',', 'supportive', 'and', 'empathetic', 'from', 'initial', 'contact', 'to', 'exchange', '.', 'Thank', 'you', '!'], ['I', 'only', 'have', 'the', 'highest', 'praise', 'for', 'Emmanuel', 'from', 'St', 'Trinity', 'Property', 'Group', '.', 'His', 'level', 'of', 'professionalism', 'and', 'c

In [113]:
pip install vaderSentiment

In [115]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [120]:
analyzer = SentimentIntensityAnalyzer()
for sentence in reviewList:
  vs = analyzer.polarity_scores(sentence)
  print("{:-<65} {}".format(sentence, str(vs)))

Emmanuel was extremely helpful and supportive throughout the entire property purchase process. As a first home buyer, there were many things I was unaware of and Emmanuel took the time to explain how it all works and how to assess properties to truly make the most of my investment. Emmanuel even went out of his way to help me investigate properties beyond the ones I originally contacted him about so I could feel confident in my decision - he was patient, supportive and empathetic from initial contact to exchange. Thank you! {'neg': 0.017, 'neu': 0.775, 'pos': 0.208, 'compound': 0.9598}
I only have the highest praise for Emmanuel from St Trinity Property Group. His level of professionalism and client service is second to none! I first met Emmanuel with a very clear view of the apartments I wanted to inspect for purchase consideration. However, Emmanuel still took the time to understand my needs and then suggested 3 other apartments I should also look at, in addition to the apartments I 

In [122]:
print(vs)

KeyError: ignored